This notebook is used to clean and prepare the genre label data for analysis.

To Do:

- [ ] Note: Katharina_Nuttall has no genres, but on wikipedia there are genres listed
- [ ] clean up names of some genres, such as 'R&amp;B'
- [ ] the following appears as a genre label: 'R&amp;B/Dance music (page does not exist)'; this appears to be another reason not to scrape the link for the genre

## Outline of Cleaning:

- [x] remove artists for which 'retrieved' value is 'none'
- [x] remove the url prefix from the retrieved artist names 
- [x] replace ' ' in the artist column with '_'
- [x] remove the '(singer)', '(rapper)', '(musician)' designation from the 'retrieved' column
- [x] remove the artists for which the retrieved-artist != searched-artist. 
    - inspect mismatches to look for typos and different versions
- [x] convert genre column values into lists of strings
- [x] remove old columns
- [x] extract unique genres as a list
- [ ] select 1% sample to verify gender

In [1]:
import numpy as np
np.random.seed(23)
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import re

from datetime import datetime

In [2]:
%ls -lt /Users/Daniel/Code/Genre/data/genre_lists/

total 0
drwxr-xr-x   7 Daniel  staff  224 Apr 27 07:34 data_ready_for_model/
drwxr-xr-x  17 Daniel  staff  544 Apr 27 07:33 data_to_be_cleaned/


### Data Sets

The file singers_gender.csv is from Kaggle and lists music artists and their gender. This is our starting point. It is augmented using the lists of women artists. Genre and network info will be generated by scraping databases. For now we are focusing in Wikipedia.

In [3]:
# kaggle_data = pd.read_csv('singers_gender.csv', encoding = 'latin-1')

In [4]:
# kaggle_data.shape

### Load the data to be cleaned:

Current: wiki-kaggle_genres_rough.csv

- This will be replaced by the fully scraped set
- The full set needs to be cleaned

Add in a converter that splits the genre list on commas:
https://stackoverflow.com/questions/32742976/how-to-read-a-column-of-csv-as-dtype-list-using-pandas

In [5]:
data = pd.read_csv('../../data/genre_lists/data_to_be_cleaned/wiki-kaggle_genres_rough_gender.csv', header = 0)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [6]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,https://en.wikipedia.org/wiki/Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,https://en.wikipedia.org/wiki/Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,https://en.wikipedia.org/wiki/Tha_Realest,['Hip hop music']
3,Mr. Lif,male,https://en.wikipedia.org/wiki/Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,https://en.wikipedia.org/wiki/Clarice_Mayne,"['Music hall', 'Variety theatre']"


In [7]:
data[data.artist == 'Katharina Nuttall']

,artist,gender,retrieved,genre
20765,Katharina Nuttall,female,https://en.wikipedia.org/wiki/Katharina_Nuttall,[]


In [8]:
data.shape

(23177, 4)

In [9]:
data.isnull().sum()

artist       0
gender       0
retrieved    0
genre        0
dtype: int64

For how many artists is the scraped genre 'none':

In [10]:
(data.genre == 'none').sum()

7704

For how many artists is the 'retrieved' value 'none':

In [11]:
(data.retrieved == 'none').sum()

7704

Take a glance at artist and retrieved values to determine necessary cleaning:

In [12]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

for n in rints:
    print('artist: {}        retrieved: {}'.format(data.artist.iloc[n], data.retrieved.iloc[n]))

artist: Alexandra Deshorties        retrieved: none
artist: Mark Vieha        retrieved: none
artist: Aidan Knight        retrieved: https://en.wikipedia.org/wiki/Aidan_Knight
artist: Butterscotch        retrieved: https://en.wikipedia.org/wiki/Butterscotch_(singer)
artist: Jani Lauzon        retrieved: none
artist: Tom Brosseau        retrieved: https://en.wikipedia.org/wiki/Tom_Brosseau
artist: Chris Medina        retrieved: https://en.wikipedia.org/wiki/Chris_Medina
artist: Lobo        retrieved: https://en.wikipedia.org/wiki/Lobo_(musician)
artist: Nick Loren        retrieved: none
artist: Álex Lora        retrieved: none
artist: D'Angelo        retrieved: https://en.wikipedia.org/wiki/D'Angelo
artist: Jinder        retrieved: none
artist: Steve Hart        retrieved: https://en.wikipedia.org/wiki/Steve_Hart_(singer)
artist: Patrick Bourque        retrieved: https://en.wikipedia.org/wiki/Patrick_Bourque
artist: Natalia Yanchak        retrieved: none


Notes on Retrieved:

- Underscore is used to separate parts of the name
- '.' are allowed in names 
- '(singer)' and '(musician)' are sometimes included and need to be stripped (probably to distinguish from othe people in wikipedia)
- double quotes are allowed in names
- hyphens appear

### Remove artists for which 'retrieved' value is 'none'

Convert none to null:

In [13]:
data['retrieved'] = data['retrieved'].replace('none', np.nan)

In [14]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,https://en.wikipedia.org/wiki/Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,https://en.wikipedia.org/wiki/Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,https://en.wikipedia.org/wiki/Tha_Realest,['Hip hop music']
3,Mr. Lif,male,https://en.wikipedia.org/wiki/Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,https://en.wikipedia.org/wiki/Clarice_Mayne,"['Music hall', 'Variety theatre']"


In [15]:
data.isnull().sum()

artist          0
gender          0
retrieved    7704
genre           0
dtype: int64

Drop rows with nulls:

In [16]:
data.dropna(axis = 0, inplace = True)

In [17]:
data.shape

(15473, 4)

In [18]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,https://en.wikipedia.org/wiki/Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,https://en.wikipedia.org/wiki/Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,https://en.wikipedia.org/wiki/Tha_Realest,['Hip hop music']
3,Mr. Lif,male,https://en.wikipedia.org/wiki/Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,https://en.wikipedia.org/wiki/Clarice_Mayne,"['Music hall', 'Variety theatre']"


## Remove the prefix from the 'retrieved' values

In [19]:
"""This function extracts artist name from the url.
Apply it to the 'retrieved' values."""
def retrieved_artist(text):
    try:
        retrieved = text
        p = re.compile(r'(https://en.wikipedia.org/wiki/)(.*)')
        result = re.match(p, retrieved)
        return result.group(2)
    except:
        if text == 'none':
            return 'none'
    else:
        return 'None'

In [20]:
data['retrieved'] = data.retrieved.apply(retrieved_artist)

In [21]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,Tha_Realest,['Hip hop music']
3,Mr. Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']"


## Replace spaces with _ in the artist column:

In [22]:
"""This function replaces white space in the values of
the column artist with an underscore."""
def underscore(text):
    try:
        split_name = text.split(' ')
        return '_'.join(split_name)  
    except:
        return 'error'

In [23]:
data['artist'] = data.artist.apply(underscore)

In [24]:
data.head()

,artist,gender,retrieved,genre
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha_Realest,male,Tha_Realest,['Hip hop music']
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']"


## Remove the \_(singer) type designation from retrieved

In [25]:
"""This function uses re. to remove any parenthetical designations
form the retrieved artist name"""
def remove_designation(text):
    designations = [r'_\(singer\)', r'_\(musician\)', r'_\(rapper\)', r'_\(band\)', r'_\(composer\)', r'_\(music_producer\)', r'_\(singer-songwriter\)' ]
    x = text
    for des in designations:
        if re.search(des, x):
            x = re.sub(r'{}'.format(des),'',text)
    return x

Apply the function:

In [26]:
data['retrieved_clean'] = data.retrieved.apply(remove_designation)

Take a glance at artist and retrieved_clean values:

In [27]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

data[['artist','retrieved_clean']].iloc[rints]

,artist,retrieved_clean
80,Frankie_Avalon,Frankie_Avalon
3756,Lily_Allen,Lily_Allen
18403,Lana_Lane,Lana_Lane
17969,Jeanne_Sagan,Jeanne_Sagan
7946,Nigel_Wright,Nigel_Wright
16949,Béla_Fleck,Béla_Fleck
19249,Ewen_Henderson,Ewen_Henderson
14680,Connie_Cato,Connie_Cato
17360,John_Dee_Holeman,John_Dee_Holeman
8560,Dave_Zegarac,Dave_Zegarac


### Mark the rows for which retrieved_clean is different from artist

In [28]:
"""This function takes a pair of strings and checks
if they are equivalent (case insensitive)

.casefold is used to be case insensitive; 
still might have problems on some characters"""

def verify_artist(x,y):
    if x.casefold() == y.casefold(): 
        return 1
    else:
        return 0

Introduce a mismatch just to make sure we can properly remove these:

In [29]:
# use an iloc index larger than the size of the original dataframe
#data.iloc[data.shape[0]+1] = ['test','test_wrong','universal','test_wrong']

Apply the function:

In [30]:
data['match'] = (data.artist.apply(lambda x: x.casefold()) != data.retrieved_clean.apply(lambda x: x.casefold())).astype('int64')

In [31]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus...",Billy_Joe_Royal,0
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']",Inés_Gaviria,0
2,Tha_Realest,male,Tha_Realest,['Hip hop music'],Tha_Realest,0
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli...",Mr._Lif,0
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']",Clarice_Mayne,0


In [32]:
data.match.sum()

0

Now remove artists where retrieved_clean doesn't match artist:

In [33]:
data = data[data.match == 0]

In [34]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus...",Billy_Joe_Royal,0
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']",Inés_Gaviria,0
2,Tha_Realest,male,Tha_Realest,['Hip hop music'],Tha_Realest,0
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli...",Mr._Lif,0
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']",Clarice_Mayne,0


Now the remaining artists are verified and have non-null genre label. 

### Genre Labels

Each value of the genre column is a _string_ of comma separated genre labels using the spotify abbreviations. We want to convert it to a _list_ of strings.

In [35]:
"""This function takes in a string of the form
appearing in the genrelist of the dataframe.
It strips the square brackets and extra quotes and
returns a list of strings where each string is a genre label."""
def genrelist(string):
    string = string.strip("[").strip("]").replace("'","")
    L = [s for s in string.split(',')]
    L_new = []
    for x in L:
        L_new.append(x.replace(" ","_").lstrip("_").rstrip("_"))
    while (str("") in L_new):
        L_new.remove("")
    return L_new

Now we apply it to the whole column and put the lists in a new column:

In [36]:
data['genrelist']= data['genre'].apply(genrelist)

In [37]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match,genrelist
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus...",Billy_Joe_Royal,0,"[Rock_and_roll, Country_music, Gospel_music]"
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']",Inés_Gaviria,0,"[Latin_pop, Pop_rock]"
2,Tha_Realest,male,Tha_Realest,['Hip hop music'],Tha_Realest,0,[Hip_hop_music]
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli...",Mr._Lif,0,"[Hip_hop_music, Alternative_hip_hop, Political..."
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']",Clarice_Mayne,0,"[Music_hall, Variety_theatre]"


In [38]:
data.genrelist.iloc[0]

['Rock_and_roll', 'Country_music', 'Gospel_music']

### Remove all artists with null values for genre :

In [39]:
data.isnull().sum(axis = 0)

artist             0
gender             0
retrieved          0
genre              0
retrieved_clean    0
match              0
genrelist          0
dtype: int64

In [40]:
data = data[data['genrelist'].notnull()]

In [41]:
data.isnull().sum(axis = 0)

artist             0
gender             0
retrieved          0
genre              0
retrieved_clean    0
match              0
genrelist          0
dtype: int64

In [42]:
data.shape

(15473, 7)

### Create column with length of genre lists:

In [43]:
data['genrelist_length'] = data.genrelist.apply(lambda x: len(x))

In [44]:
data.shape

(15473, 8)

### Remove artists for which the genre list is empty:

In [45]:
data = data[data.genrelist_length > 0]

In [46]:
data.shape

(14615, 8)

We lost around 800 artists because they had empty genrelists. I checked a few of these and they have genres listed on Wikipedia. 

- [ ] check the scraping code to retain these?

Remove old columns:

In [47]:
data.columns

Index(['artist', 'gender', 'retrieved', 'genre', 'retrieved_clean', 'match',
       'genrelist', 'genrelist_length'],
      dtype='object')

In [48]:
data.drop(['retrieved','genre','retrieved_clean', 'match'], axis = 1, inplace = True)

In [49]:
data.head()

,artist,gender,genrelist,genrelist_length
0,Billy_Joe_Royal,male,"[Rock_and_roll, Country_music, Gospel_music]",3
1,Inés_Gaviria,female,"[Latin_pop, Pop_rock]",2
2,Tha_Realest,male,[Hip_hop_music],1
3,Mr._Lif,male,"[Hip_hop_music, Alternative_hip_hop, Political...",3
4,Clarice_Mayne,female,"[Music_hall, Variety_theatre]",2


In [50]:
data.shape

(14615, 4)

Remove any artists without gender value

In [51]:
data.gender.isnull().sum()

0

In [52]:
data.gender.unique()

array(['male', 'female'], dtype=object)

### Extracting the unique genre labels:

First make a list of the genrelists:

In [53]:
genre_list = data.genrelist.values.tolist()
genre_list = [x for y in genre_list for x in y]
genre_list = list(set(genre_list))

In [54]:
genre_list[:5]

['Outsider_music',
 'Mexican_cumbia',
 'Novelty_songs',
 'Heavy_metal_music',
 'St._Louis_blues_(music)']

In [55]:
len(genre_list)

1760

In [56]:
print('There are {} artists with genre and binary-gender labels.'.format(data.shape[0]))
print('There are {} unique genre labels.'.format(len(genre_list)))

There are 14615 artists with genre and binary-gender labels.
There are 1760 unique genre labels.


Check to see if there are artists with len(genrelist) = 0

In [57]:
data[data.genrelist.apply(lambda x: True if len(x) == 0 else False)]

,artist,gender,genrelist,genrelist_length


## Select 1% of artists to verify gender manually.

In [58]:
data.shape

(14615, 4)

In [59]:
data_male = data[data.gender == 'male']
data_female = data[data.gender == 'female']

In [60]:
tot = data.shape[0]
m = data_male.shape[0]
f = data_female.shape[0]
print('{} total artists'.format(tot))
print('{} female artists, or {:0.0f}%'.format(f, 100*f/(f+m)))
print('{} male artists, or {:0.0f}%'.format(m, 100*m/(f+m)))

14615 total artists
4526 female artists, or 31%
10089 male artists, or 69%


We will verify the same number of female as male artists. Let 
- $p$ be the fraction of artists to verify,
- $p_f$ be the fraction of female artists to verify
- $p_m$ be the fraction of male artists to verify

We want two conditions satisfied:

$$ p_f \cdot f = p_m \cdot m $$
$$ p_f \cdot f + p_m \cdot m = p \cdot (f+m)$$

These encode the requirements that
- we verify equal numbers of male and female artists
- the fractino of artists we verify is $p$

Solving for $p_f$ and $p_m$ leads to 

$$ p_f = \frac{p}{2f}(f+m)$$
$$ p_m = \frac{p}{2m}(f+m)$$

Define the functions for $p_f$ and $p_m$:

In [61]:
def pf(p,f,m):
    return p*(m+f)/(2*f)

def pm(p,f,m):
    return p*(m+f)/(2*m)

Substituting in $p=.01$, $f = 4864$, and $m = 10609$ leads to 

In [62]:
p = .01
p_f, p_m = pf(p,f,m), pm(p,f,m)

We select samples from the data:

In [63]:
sample_female = data_female.sample(frac = p_f)
sample_male = data_male.sample(frac = p_m)

In [64]:
sample_male.shape[0], sample_female.shape[0]

(73, 73)

In [65]:
sample_fm = pd.concat([sample_female,sample_male])

In [66]:
sample_size = sample_fm.shape[0]

In [67]:
sample_fm = sample_fm.sample(sample_size)

In [68]:
sample_fm.head()

,artist,gender,genrelist,genrelist_length
7603,Karan_Casey,female,"[Celtic_music, Folk_music]",2
2252,Michelle_Chamuel,female,"[Pop_music, Electropop, Electronica]",3
20702,Sa-Fire,female,"[Pop_music, Dance_music]",2
19523,Colette_Carr,female,"[Pop_music, Pop-rap, Electropop, Indie_pop, Eu...",5
2860,"James_""Stump""_Johnson",male,[St._Louis_blues_(music)],1


In [69]:
tom_sample = sample_fm[:73]
dan_sample = sample_fm[73:]
tom_sample.shape, dan_sample.shape

((73, 4), (73, 4))

In [70]:
dan_sample.head(20)

,artist,gender,genrelist,genrelist_length
10844,Bill_Kirchen,male,"[Ameripolitan, Rockabilly, Rock_and_roll, Coun...",8
2798,Olivia_O'Brien,female,"[Pop_music, Trip_hop]",2
10180,Rissi_Palmer,female,"[Country_music, R&amp;B, Southern_soul]",3
7250,Fantasia_Barrino,female,"[Contemporary_R&amp;B, Pop_music, Soul_music, ...",5
3766,Davey_Pattison,male,"[Hard_rock, Blues-rock, Heavy_metal_music, Blu...",5
5920,Nona_Hendryx,female,"[Funk, Soul_music, Pop_music, Rhythm_and_blues...",8
2936,Dave_Sturt,male,"[Progressive_rock, Symphonic_rock, World_Music]",3
17864,Gayle_McCormick,female,"[Rock_music, Blues]",2
17175,Gwyneth_Herbert,female,"[Singer-songwriter, Jazz, Musical_theatre]",3
17677,Stacey_Solomon,female,[Pop_music],1


- [ ] The following artists have popped up as having zero labels in the sample. How are they getting through the len >0 filter?

In [71]:
zero_labels = ['Katharina Nuttall', 'Aebh_Kelly', 'Ronnie_Sessions', 'John_Dankworth']

# SWITCH 'now' to datetime when have cleaned and verified dataset

Export data for further use:

In [72]:
today = datetime.today()
#now = today.strftime('%Y-%m-%d-%H-%M')
now = 'temp'

In [73]:
data.to_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_genres_gender_cleaned_{}.csv'.format(now))